In [17]:
import os
import re
import itertools
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks
from tabulate import tabulate

### Reading the data from 1st and 2nd rounds

In [18]:
datapath1 = '../data/img_labeling_1st_round/'
datapath2 = '../data/img_labeling_2nd_round/'

In [19]:
df_round1 = pd.read_pickle(os.path.join(datapath1,'df_labeling_round1.pkl'))
df_round2 = pd.read_pickle(os.path.join(datapath2,'df_labeling_round2.pkl'))

In [20]:
df_round1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 451
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        5880 non-null   int64         
 1   user        5880 non-null   object        
 2   date        5880 non-null   datetime64[ns]
 3   image name  5880 non-null   object        
 4   id_image    5880 non-null   int16         
 5   class       5880 non-null   object        
 6   round       5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 333.0+ KB


In [22]:
df_round2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 391
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5880 non-null   int64         
 1   user               5880 non-null   object        
 2   date               5880 non-null   datetime64[ns]
 3   image name         5880 non-null   object        
 4   id_image           5880 non-null   int16         
 5   class              5880 non-null   object        
 6   image_appearances  5880 non-null   int64         
 7   round              5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(3), object(3)
memory usage: 379.0+ KB


In [23]:
df_round2.drop('image_appearances', inplace=True, axis=1)
df = pd.concat([df_round1, df_round2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11760 entries, 0 to 391
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        11760 non-null  int64         
 1   user        11760 non-null  object        
 2   date        11760 non-null  datetime64[ns]
 3   image name  11760 non-null  object        
 4   id_image    11760 non-null  int16         
 5   class       11760 non-null  object        
 6   round       11760 non-null  int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 666.1+ KB


In [24]:
df.head()

,task,user,date,image name,id_image,class,round
0,1,Amelie,2020-08-09,203-resource_document_zuiderzeemuseum_B001601_...,87,Non-Fruits,1
1,1,Amelie,2020-08-09,285-gam19649_1.jpeg,232,Fruits,1
2,1,Amelie,2020-08-09,07101-O_389_1.jpeg,6,Fruits,1
3,1,Amelie,2020-08-09,07101-O_927_1.jpeg,12,Fruits,1
4,1,Amelie,2020-08-09,07101-O_957_1.jpeg,13,Non-Fruits,1


### Analysing the [inter-annotator agreement](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) on the results

In [12]:
l1 = df_labeling.user.unique()
iter_users = list(itertools.product(l1,l1))
df_iaa = pd.DataFrame(index=l1, columns=l1)

In [13]:
for task in df_labeling.task.unique():
    for user1,user2 in iter_users:
        classesA = df_labeling.loc[(df_labeling.user == user1) & (df_labeling.task == task),['id_image', 'class']]
        classesA.sort_values(by=['id_image'], inplace=True)

        classesB = df_labeling.loc[(df_labeling.user == user2) & (df_labeling.task == task),['id_image', 'class']]
        classesB.sort_values(by=['id_image'], inplace=True)
        
        classesAB = pd.merge(classesA, classesB, on=['id_image'])
        classesAB.drop_duplicates(subset='id_image', keep = 'first', inplace=True) 
        classesAB.drop('id_image', axis=1, inplace=True)
        classesAB.dropna(inplace=True)

        agreement = cks(classesAB['class_x'], classesAB['class_y'])
        df_iaa.loc[user1,user2] = f'{agreement:.3f}/({len(classesAB)})'  
        df_iaa.index.name = f'Task_{task}'
    print(tabulate(df_iaa, headers='keys', tablefmt='psql'))
    print()

+-----------+-------------+-------------+-------------+-------------+-------------+
| Task_1    | Amelie      | Gerda       | Marcos      | Renato      | Yalemisew   |
|-----------+-------------+-------------+-------------+-------------+-------------|
| Amelie    | 1.000/(392) | 0.928/(392) | 0.898/(392) | 0.908/(392) | 0.898/(392) |
| Gerda     | 0.928/(392) | 1.000/(392) | 0.882/(392) | 0.907/(392) | 0.861/(392) |
| Marcos    | 0.898/(392) | 0.882/(392) | 1.000/(392) | 0.913/(392) | 0.928/(392) |
| Renato    | 0.908/(392) | 0.907/(392) | 0.913/(392) | 1.000/(392) | 0.903/(392) |
| Yalemisew | 0.898/(392) | 0.861/(392) | 0.928/(392) | 0.903/(392) | 1.000/(392) |
+-----------+-------------+-------------+-------------+-------------+-------------+

+-----------+-------------+-------------+-------------+-------------+-------------+
| Task_2    | Amelie      | Gerda       | Marcos      | Renato      | Yalemisew   |
|-----------+-------------+-------------+-------------+-------------+------

In [14]:
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5880 entries, 0 to 391
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   task               5880 non-null   int64         
 1   user               5880 non-null   object        
 2   date               5880 non-null   datetime64[ns]
 3   image name         5880 non-null   object        
 4   id_image           5880 non-null   int16         
 5   class              5880 non-null   object        
 6   image_appearances  5880 non-null   int64         
dtypes: datetime64[ns](1), int16(1), int64(2), object(3)
memory usage: 333.0+ KB


In [15]:
df_labeling['round'] = 2

In [16]:
df_labeling.to_pickle('../data/img_labeling_2nd_round/df_labeling_round2.pkl')